In [1]:
#pip install gpxpy

In [2]:
import gpxpy
import gpxpy.gpx
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [ ]:
df_act=pd.read_csv("bcs.csv")
df_act= df_act[df_act["Activity Type"] == "Backcountry Skiing"]
df_act

In [ ]:
act_list = df_act["Activity ID"]
df_list=[]
for act in act_list:
    gpx_name = "act_data/activity_" + str(act) + ".gpx"
    with open(gpx_name, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    route_info = []

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                route_info.append({
                    'latitude': point.latitude,
                    'longitude': point.longitude,
                    'elevation': point.elevation,
                    'time':point.time
                })
    df = pd.DataFrame(route_info)
    df['time'] = pd.to_datetime(df['time'])
    df['time'] = df['time'].dt.tz_localize(None)
    df.set_index('time', inplace=True)
    def seconds_to_hm(seconds):
        return str(datetime.timedelta(seconds=seconds))
    # calculate the time difference in seconds
    df['training_time'] = (df.index - df.index[0]).total_seconds()
    # set the first value to zero
    df['training_time'] = df['training_time'] - df['training_time'].iloc[0]
    df['tt_h'] = df['training_time'].apply(seconds_to_hm)
    df.set_index('tt_h', inplace=True)
    df= df[["elevation"]]
    df.columns=[str(act)]
    df_list.append(df)
df_concat = pd.concat(df_list,axis =1)
df_concat = df_concat.sort_index()
df_concat = df_concat.fillna(method="bfill")
df_concat

In [ ]:
# Set the style
sns.set_style('darkgrid')

# Plot the data
fig, ax = plt.subplots(figsize=(16, 10))
df_concat.plot(ax=ax)
ax.set_xlabel('Time')
ax.set_ylabel('Elevation')
ax.set_title('Elevation over Time')

plt.show()

In [6]:
df_act= df_act[df_act["Location Name"] == "Klæbu"]
act_list = df_act["Activity ID"]
df_list=[]
for act in act_list:
    gpx_name = "act_data/activity_" + str(act) + ".gpx"
    with open(gpx_name, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)
    route_info = []
    for track in gpx.tracks:
            for segment in track.segments:
                for point in segment.points:
                    route_info.append({
                        'latitude': point.latitude,
                        'longitude': point.longitude,
                        'elevation': point.elevation,
                        'time':point.time
                    })
    route_df = pd.DataFrame(route_info)
    df_list.append(route_df)

In [ ]:
colors = ['red','blue','green','purple','orange','darkred','pink']
fg = folium.FeatureGroup(name='lines')
map = folium.Map(location=[63.2634, 10.40], zoom_start=15)
# create polyline for each row in dataframe
for c, route_df in enumerate(df_list):
    cord_list=[]
    for i, row in route_df.iterrows():
        coords = [row['latitude'], row['longitude']]
        cord_list.append(coords)
    
    my_PolyLine=folium.PolyLine(locations=cord_list,color=colors[c], weight=5, opacity=0.5)
    map.add_child(my_PolyLine)
map
